# STEAM VIDEO GAMES RECOMMENDATION SYSTEM

# BUSINESS PROBLEM

Steam is the largest video game hosting platform in the market with a large caleloge full of games. With access to such a variety of games, sometimes users may feel overwelmed, or feel unsure with where to start, there for it is absolutely vital that both customers new and old are provided with recommendended suitable games that align with their specific preferences. 
<br><br>
Therefore for this project i decided to create a recommendation system for steam games to help users find suitable game recommendations that they would be interested in. My goal for this project is to provide more accurate predictions regarding providing a user with recommendations.
<br><br>
For this project i will be creating a content and collaborative based recommendation model that provides video game recommendations to steam users. This project will utlize Steam's API, SteamSpy's API along with data sets sourced from kaggle in order to create a recommendation system. 
<br><br>
The aim of this project is to improve steam's current recommendation system to provide users with much more accurate recommendations.

# DATA PREPARATION

### Importing necessary packages

In [169]:
# importing necessary packages
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import json
import requests
from bs4 import BeautifulSoup

from sklearn.preprocessing import StandardScaler 
from sklearn import preprocessing

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Obtaining the API key

In [170]:
# function that obtains api key from folder
def get_keys(path):
    ''' this function obtains the api key'''
    with open(path) as f:
        return json.load(f)

In [171]:
# getting the api key
keys = get_keys("/Users/Visitor/Documents/Flatiron/Capstone/.secret/steam_api.json")

In [172]:
# getting and assigning the api key to a variable
api_key = keys['api_key']

# Generating requests from Steam's API

I will be using steams API to gain information regarding player informatoin, their recently played games, friends lists, users statistics for a particular game and a list of all steam games.

From Steam's api the following data will be used:
<li>GetPlayerSummaries
<li>GetUserStatsForGame
<li>GetRecentlyPlayedGames
<li>GetOwnedGames
<li>GetFriendList
<li>GetAppList

This data is obtained from: https://steamcommunity.com/dev

### Player summaries (GetPlayerSummaries)

Player summaries provides their steam id, along with other user information.

In [173]:
# getting requests from the api for the player summaries
resp = requests.get("http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key=B8DD767E6A0BF4F61B05D6E5483C29A9&steamids=76561197960435530")

In [174]:
# checking if the request was sucessful
resp.status_code == requests.codes.ok

True

In [175]:
#filtering the data
resp.json()['response']['players']

[{'steamid': '76561197960435530',
  'communityvisibilitystate': 3,
  'profilestate': 1,
  'personaname': 'Robin',
  'profileurl': 'https://steamcommunity.com/id/robinwalker/',
  'avatar': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/f1/f1dd60a188883caf82d0cbfccfe6aba0af1732d4.jpg',
  'avatarmedium': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/f1/f1dd60a188883caf82d0cbfccfe6aba0af1732d4_medium.jpg',
  'avatarfull': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/f1/f1dd60a188883caf82d0cbfccfe6aba0af1732d4_full.jpg',
  'avatarhash': 'f1dd60a188883caf82d0cbfccfe6aba0af1732d4',
  'personastate': 0,
  'realname': 'Robin Walker',
  'primaryclanid': '103582791429521412',
  'timecreated': 1063407589,
  'personastateflags': 0,
  'loccountrycode': 'US',
  'locstatecode': 'WA',
  'loccityid': 3961}]

In [176]:
# creating a data frame using the obtained information
df = pd.DataFrame.from_dict(resp.json()['response']['players'])

In [177]:
# displaying the data frame
df.head()

,steamid,communityvisibilitystate,profilestate,personaname,profileurl,avatar,avatarmedium,avatarfull,avatarhash,personastate,realname,primaryclanid,timecreated,personastateflags,loccountrycode,locstatecode,loccityid
0,76561197960435530,3,1,Robin,https://steamcommunity.com/id/robinwalker/,https://steamcdn-a.akamaihd.net/steamcommunity...,https://steamcdn-a.akamaihd.net/steamcommunity...,https://steamcdn-a.akamaihd.net/steamcommunity...,f1dd60a188883caf82d0cbfccfe6aba0af1732d4,0,Robin Walker,103582791429521412,1063407589,0,US,WA,3961


### Friend's list (GetFriendList)

Friends list, generates a list of all friends for a specific user. Having this information will further help identify which games to recommend a user as it could be the case that these friends could have similar preferences, and therefore may like similar games

In [178]:
# getting requests from the api for users friend lists
resp = requests.get("http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key=B8DD767E6A0BF4F61B05D6E5483C29A9&steamid=76561197960435530&relationship=friend")

In [179]:
# checking if the request was sucessful
resp.status_code == requests.codes.ok

True

In [180]:
# filtering the data
# resp.json()['friendslist']['friends']

In [181]:
# creating a data frame from the obtained data
df = pd.DataFrame.from_dict(resp.json()['friendslist']['friends'])

In [182]:
# displaying the newly created data frame
df.head()

,steamid,relationship,friend_since
0,76561197960265731,friend,0
1,76561197960265738,friend,0
2,76561197960265740,friend,0
3,76561197960265744,friend,1585508613
4,76561197960265747,friend,0


### List of all games (GetAppList)

In [183]:
# obtaining requests from the api to get a list of steam games
resp = requests.get("https://api.steampowered.com/ISteamApps/GetAppList/v2/")

In [184]:
# checking if the request was sucessful
resp.status_code == requests.codes.ok

True

In [185]:
#filtering the data
#resp.json()['applist']['apps']

In [186]:
# creating a data frame with the obtained data
df_games = pd.DataFrame.from_dict(resp.json()['applist']['apps'])

In [187]:
# displaying the data frame
df_games.head()

,appid,name
0,216938,Pieterw test app76 ( 216938 )
1,660010,test2
2,660130,test3
3,479990,Dungeons & Darkness
4,479220,Sector Six Demo


# Generating requests from SteamSpy

As i was unable to source all of the game information from steam's api, i will be using steamspy, which contains an entire list of steam games, their app id, genres and all other relevent information.

This data is obtained from: https://steamspy.com/

### Using steamspy to request steam's game information

In [188]:
# requesting all game data from steamspy
resp = requests.get("http://steamspy.com/api.php?request=all")

In [189]:
# converting the request to json format
dic_app_user = resp.json()

In [190]:
# obtaining the keys of the dictionary
lst_app_id = dic_app_user.keys()

In [191]:
# checking length
len(dic_app_user)

35900

In [192]:
# checking if the request was sucessful
resp.status_code == requests.codes.ok

True

In [193]:
# filtering the data
# resp.json()

In [194]:
# converting the dictionary to a data frame
df = pd.DataFrame.from_dict(resp.json())

In [195]:
# displaying the data frame
df

,570,730,578080,440,304930,230410,271590,359550,291550,105600,550,4000,10,444090,272060,236390,240,238960,218620,49520,227940,275390,301520,1085660,252950,291480,219990,220,360,252490,304050,620,433850,550650,386360,96000,44350,72850,8930,417910,80,381210,431960,292030,582010,218230,70,755790,320,370910,...,763340,865810,804910,804540,541310,736240,750200,751270,761720,723360,593150,599490,743920,785790,611820,652730,789710,729290,705000,591020,646240,858730,651490,654200,706550,587030,46470,852,701360,18310,17760,497960,721460,859120,512720,848400,643930,553140,201930,675330,778140,781150,874630,565140,565100,523900,1000130,514720,1000600,655270
appid,570,730,578080,440,304930,230410,271590,359550,291550,105600,550,4000,10,444090,272060,236390,240,238960,218620,49520,227940,275390,301520,1085660,252950,291480,219990,220,360,252490,304050,620,433850,550650,386360,96000,44350,72850,8930,417910,80,381210,431960,292030,582010,218230,70,755790,320,370910,...,763340,865810,804910,804540,541310,736240,750200,751270,761720,723360,593150,599490,743920,785790,611820,652730,789710,729290,705000,591020,646240,858730,651490,654200,706550,587030,46470,852,701360,18310,17760,497960,721460,859120,512720,848400,643930,553140,201930,675330,778140,781150,874630,565140,565100,523900,1000130,514720,1000600,655270
name,Dota 2,Counter-Strike: Global Offensive,PLAYERUNKNOWN'S BATTLEGROUNDS,Team Fortress 2,Unturned,Warframe,Grand Theft Auto V,Tom Clancy's Rainbow Six Siege,Brawlhalla,Terraria,Left 4 Dead 2,Garry's Mod,Counter-Strike,Paladins,Serena,War Thunder,Counter-Strike: Source,Path of Exile,PAYDAY 2,Borderlands 2,Heroes & Generals,Guacamelee! Super Turbo Championship Edition,Robocraft,Destiny 2,Rocket League,Warface,Grim Dawn,Half-Life 2,Half-Life Deathmatch: Source,Rust,Trove,Portal 2,Z1 Battle Royale,Black Squad,SMITE,The Tiny Bang Story,GRID 2,The Elder Scrolls V: Skyrim,Sid Meier's Civilization V,Street Warriors Online,Counter-Strike: Condition Zero,Dead by Daylight,Wallpaper Engine,The Witcher 3: Wild Hunt,MONSTER HUNTER: WORLD,PlanetSide 2,Half-Life,Ring of Elysium,Half-Life 2: Deathmatch,Kathy Rain,...,Brave Hand,Waifu Fight Dango Style,Mine Seeker,Greedy Developer's Cash Grab,Tiny Thor,The Revolt: Awakening,AWAY: The Survival Series,BioEntity,Can you eat by yourself,Asunder,Ooblets,Bounty Killer,Choo-Choo! The Train Rides!,WHAT THE GOLF?,Xenosis: Alien Infection,Z-Aftershock,BoyAndLabyrinth,Hidden Cubes,OctorSpace,山贼,Bacon May Die,Yoke Light,No Longer Home,Seven: Reboot,Kaya,Bunker Busters Steamworks Test,Grotesque Tactics: Evil Heroes - Dev,ValveTestApp852,Fate Crawler,Spectraball - Demo,,Legends of Callasia Demo,Xeno Time Inception,That Tiny Spaceship,Velocidevorium,Simian Rising,拯救大魔王3 Falsemen3,Green Mirror,Jamestown IGF,Space Crawl,BRIKS 2,Gene Rain,M.A.D. Cliff - All Quiet On The Bridge,Chimeras: Tune of Revenge Collector's Edition,Subliminal Realms: The Masterpiece Collector's...,Fated Souls 2,Cube Defender,Dark Tales: Edgar Allan Poe's The Masque of th...,The ScreaMaze,EPONYMOUS
developer,Valve,"Valve, Hidden Path Entertainment",PUBG Corporation,Valve,Smartly Dressed Games,Digital Extremes,Rockstar North,Ubisoft Montreal,Blue Mammoth Games,Re-Logic,Valve,Facepunch Studios,Valve,Evil Mojo Games,Senscape,Gaijin Entertainment,Valve,Grinding Gear Games,OVERKILL - a Starbreeze Studio.,"Gearbox Software, Aspyr (Mac), Aspyr (Linux)",RETO MOTO,DrinkBox Studios,Freejam,Bungie,Psyonix LLC,MY.GAMES,Crate Entertainment,Valve,Valve,Facepunch Studios,Trion Worlds,Valve,Daybreak Game Company,NS STUDIO,Titan Forge Games,Colibri Games,Codemasters Racing,Bethesda Game Studios,"Firaxis Games, Aspyr (Mac), Aspyr (Linux)",Crazy Rocks Studios,Valve,Behaviour Interactive Inc.,Wallpaper Engine Team,CD PROJEKT RED,"CAPCOM Co., Ltd.",Rogue Planet Games,Valve,Aurora Studio,Valve,Clifftop Games,...,Heart Shaped Games LLC,Enso Entertainment,Jason Crosby,Greedy Developer,Asylum Square,Siberius Studio,Breaking Walls,Sneaky Party LLC,Independent team,Dawson Fr

In [196]:
# checking the shape 
df.shape

(16, 35900)

In [197]:
# transposing the data frame to convert the columns into rows
df = df.T

In [198]:
# checking the updated data frame
df.head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount
570,570,Dota 2,Valve,Valve,,1096833,194259,0,"100,000,000 .. 200,000,000",31068,1779,1411,922,0,0,0
730,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,,3834378,533122,0,"100,000,000 .. 200,000,000",24958,1170,7370,415,0,0,0
578080,578080,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,PUBG Corporation,,781419,664521,0,"20,000,000 .. 50,000,000",23310,659,10668,224,2999,2999,0
440,440,Team Fortress 2,Valve,Valve,,627500,39186,0,"20,000,000 .. 50,000,000",9705,1751,409,577,0,0,0
304930,304930,Unturned,Smartly Dressed Games,Smartly Dressed Games,,349238,34817,0,"20,000,000 .. 50,000,000",2514,774,369,599,0,0,0


In [199]:
# checking shape
df.shape

(35900, 16)

In [200]:
# checking for any null values in this data set
df.isnull().any()

appid              False
name               False
developer          False
publisher          False
score_rank         False
positive           False
negative           False
userscore          False
owners             False
average_forever    False
average_2weeks     False
median_forever     False
median_2weeks      False
price               True
initialprice        True
discount            True
dtype: bool

In [201]:
# reseting index so that it doesnt show the appid twice
df.reset_index(drop=True, inplace=True)

In [202]:
# displaying the data frame to check the changes
df.head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount
0,570,Dota 2,Valve,Valve,,1096833,194259,0,"100,000,000 .. 200,000,000",31068,1779,1411,922,0,0,0
1,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,,3834378,533122,0,"100,000,000 .. 200,000,000",24958,1170,7370,415,0,0,0
2,578080,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,PUBG Corporation,,781419,664521,0,"20,000,000 .. 50,000,000",23310,659,10668,224,2999,2999,0
3,440,Team Fortress 2,Valve,Valve,,627500,39186,0,"20,000,000 .. 50,000,000",9705,1751,409,577,0,0,0
4,304930,Unturned,Smartly Dressed Games,Smartly Dressed Games,,349238,34817,0,"20,000,000 .. 50,000,000",2514,774,369,599,0,0,0


# Importing data frames

### Data frame with games information

This data was sourced from the steamspy api, and contains all of steams games, and their relevant game information such as scores, developers, price and other such information.

<b>Variables:</b>
<li><b>'appid':</b> unique ids of each game
<li><b>'name':</b> name of the game
<li><b>'developer':</b> developer of the game
<li><b>'publisher':</b> publisher of the game
<li><b>'score_rank':</b> publisher of the game
<li><b>'positive':</b> postive scores
<li><b>'negative':</b> negative scores
<li><b>'userscore':</b> amount of users that scores
<li><b>'owners':</b> number of people that bought the game
<li><b>'average_forever':</b> entire average play time
<li><b>'average_2weeks':</b> average play time over 2 weeks
<li><b>'median_forever':</b> entire median play time
<li><b>'median_2weeks':</b> median play time over 2 weeks
<li><b>'price':</b> price of game
<li><b>'initialprice':</b> inital price of game
<li><b>'discount':</b> discount price

In [203]:
# inital data frame with the scraped data from steamspy
df.head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount
0,570,Dota 2,Valve,Valve,,1096833,194259,0,"100,000,000 .. 200,000,000",31068,1779,1411,922,0,0,0
1,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,,3834378,533122,0,"100,000,000 .. 200,000,000",24958,1170,7370,415,0,0,0
2,578080,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,PUBG Corporation,,781419,664521,0,"20,000,000 .. 50,000,000",23310,659,10668,224,2999,2999,0
3,440,Team Fortress 2,Valve,Valve,,627500,39186,0,"20,000,000 .. 50,000,000",9705,1751,409,577,0,0,0
4,304930,Unturned,Smartly Dressed Games,Smartly Dressed Games,,349238,34817,0,"20,000,000 .. 50,000,000",2514,774,369,599,0,0,0


In [204]:
# renaming the data frame
game_info = df

### Data frame with player purchase/play information

this data set was sourced from kaggle: https://www.kaggle.com/tamber/steam-video-games 

<b>Variables:</b>
<li><b>'151603712':</b> unique ids of each game
<li><b>'The Elder Scrolls V Skyrim':</b> name of the game
<li><b>'purchase':</b> purchased or played
<li><b>'1.0':</b> hours of play, value is 1.0 if purchased
<li><b>'0':</b> 0 (this appears to be an column full of zeros and will be removed later)

In [205]:
# player purchase/play details
df_2 = pd.read_csv('steam-200k.csv')
df_2.head()

,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
0,151603712,The Elder Scrolls V Skyrim,play,273.0,0
1,151603712,Fallout 4,purchase,1.0,0
2,151603712,Fallout 4,play,87.0,0
3,151603712,Spore,purchase,1.0,0
4,151603712,Spore,play,14.9,0


The columns appear to be a row it self, so i will be bringing that information down and providing the columns with proper names.

In [206]:
# re importing the data set to bring down the column headers as a row
df_2 = pd.read_csv('steam-200k.csv', header=None)
df_2.head()

,0,1,2,3,4
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0
2,151603712,Fallout 4,purchase,1.0,0
3,151603712,Fallout 4,play,87.0,0
4,151603712,Spore,purchase,1.0,0


In [207]:
# renaming the data frame
purchase_play_info = df_2

### Data frame with games descriptions

this data set was sourced from kaggle: https://www.kaggle.com/nikdavis/steam-store-games

<b>Variables:</b>
<li><b>'steam_appid':</b> unique ids of each game
<li><b>'detailed_description':</b> detailed description of the game
<li><b>'about_the_game':</b> information about the game
<li><b>'short_description':</b> short description of the game

In [208]:
# game descriptions
df_3 = pd.read_csv('steam_description_data.csv')
df_3.head()

,steam_appid,detailed_description,about_the_game,short_description
0,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...
1,20,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...
2,30,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...
3,40,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...
4,50,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...


In [209]:
# renaming the data frame
game_desc = df_3

### Data frame with more games information

this data set was sourced from kaggle: https://www.kaggle.com/nikdavis/steam-store-games

<b>Variables:</b>
<li><b>'appid':</b> unique ids of each game
<li><b>'name':</b> name of the game
<li><b>'release_date':</b> game release date
<li><b>'english':</b> returns if the game is english or not
<li><b>'developer':</b> developer of the game
<li><b>'publisher':</b> publisher of the game
<li><b>'platforms':</b> compatible platforms
<li><b>'required_age':</b> minimum age requirements of game
<li><b>'categories':</b> game categories
<li><b>'genres':</b> game genres
<li><b>'steamspy_tags':</b> tags of game
<li><b>'achievements':</b> game achievements completed
<li><b>'positive_ratings':</b> postive ratings of the game
<li><b>'negative_ratings':</b> negative ratings of the game
<li><b>'average_playtime':</b> average playtime
<li><b>'median_playtime':</b> median playtime
<li><b>'owners':</b> amount of people who purpchased the game
<li><b>'price':</b> price

In [210]:
# information
df_4 = pd.read_csv('steam.csv')
df_4.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [211]:
# renaming the data frame
game_info_2 = df_4

# Merging the data frames

In [212]:
# displaying data frame with steamspy data
game_info.head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount
0,570,Dota 2,Valve,Valve,,1096833,194259,0,"100,000,000 .. 200,000,000",31068,1779,1411,922,0,0,0
1,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,,3834378,533122,0,"100,000,000 .. 200,000,000",24958,1170,7370,415,0,0,0
2,578080,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,PUBG Corporation,,781419,664521,0,"20,000,000 .. 50,000,000",23310,659,10668,224,2999,2999,0
3,440,Team Fortress 2,Valve,Valve,,627500,39186,0,"20,000,000 .. 50,000,000",9705,1751,409,577,0,0,0
4,304930,Unturned,Smartly Dressed Games,Smartly Dressed Games,,349238,34817,0,"20,000,000 .. 50,000,000",2514,774,369,599,0,0,0


In [213]:
# checking shape
game_info.shape

(35900, 16)

In [214]:
# checking data of the data frame containg game information
game_info_2.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [215]:
# checking shape
game_info_2.shape

(27075, 18)

In [216]:
# checking columns
game_info_2.columns

Index(['appid', 'name', 'release_date', 'english', 'developer', 'publisher', 'platforms',
       'required_age', 'categories', 'genres', 'steamspy_tags', 'achievements', 'positive_ratings',
       'negative_ratings', 'average_playtime', 'median_playtime', 'owners', 'price'],
      dtype='object')

In [217]:
# checking columns
game_info.columns

Index(['appid', 'name', 'developer', 'publisher', 'score_rank', 'positive', 'negative',
       'userscore', 'owners', 'average_forever', 'average_2weeks', 'median_forever',
       'median_2weeks', 'price', 'initialprice', 'discount'],
      dtype='object')

As both of these data frame contain all steam games, along with other relevant game information, i will be combining these two data sets to form one data set will all of game information

I will be now checking if both of these data frames are correctly assigned to their own unique app id. If the app id and their corresponding game match then i will be merging the two data frames based on the app id column. 

In [218]:
# checking a specific row
game_info[game_info['name'] == 'Dota 2']

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount
0,570,Dota 2,Valve,Valve,,1096833,194259,0,"100,000,000 .. 200,000,000",31068,1779,1411,922,0,0,0


In [219]:
# checking a specific row
game_info_2[game_info_2.name == 'Dota 2']

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
22,570,Dota 2,2013-07-09,1,Valve,Valve,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,863507,142079,23944,801,100000000-200000000,0.0


As the app ids and the name of the game match up in both of the data frames these will be merged based on the game id.

In [220]:
# dropping unessasary columns and columns that appear in both data frames
game_info.drop(['price', 'initialprice', 'discount', 'average_2weeks', 'median_2weeks', 'owners'], axis=1, inplace=True)

In [221]:
# dropping unessasary columns
game_info_2.drop(['name', 'developer', 'publisher'], axis=1, inplace=True)

In [222]:
# merging the data frames based on the appid
game_info = pd.DataFrame.merge(game_info, game_info_2,on='appid')

In [223]:
# checking the merged data frame will all of the games information
game_info.head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,average_forever,median_forever,release_date,english,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,570,Dota 2,Valve,Valve,,1096833,194259,0,31068,1411,2013-07-09,1,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,863507,142079,23944,801,100000000-200000000,0.00
1,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,,3834378,533122,0,24958,7370,2012-08-21,1,windows;mac;linux,0,Multi-player;Steam Achievements;Full controlle...,Action;Free to Play,FPS;Multiplayer;Shooter,167,2644404,402313,22494,6502,50000000-100000000,0.00
2,578080,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,PUBG Corporation,,781419,664521,0,23310,10668,2017-12-21,1,windows,0,Multi-player;Online Multi-Player;Stats,Action;Adventure;Massively Multiplayer,Survival;Shooter;Multiplayer,37,496184,487076,22938,12434,50000000-100000000,26.99
3,440,Team Fortress 2,Valve,Valve,,627500,39186,0,9705,409,2007-10-10,1,windows;mac;linux,0,Multi-player;Cross-Platform Multiplayer;Steam ...,Action;Free to Play,Free to Play;Multiplayer;FPS,520,515879,34036,8495,623,20000000-50000000,0.00
4,304930,Unturned,Smartly Dressed Games,Smartly Dressed Games,,349238,34817,0,2514,369,2017-07-07,1,windows;mac;linux,0,Single-player;Online Multi-Player;Online Co-op...,Action;Adventure;Casual;Free to Play;Indie,Free to Play;Survival;Zombies,46,292574,31482,3248,413,20000000-50000000,0.00


In [224]:
# checking the shape
game_info.shape

(25974, 24)

In [225]:
# checking all of the columns
game_info.columns

Index(['appid', 'name', 'developer', 'publisher', 'score_rank', 'positive', 'negative',
       'userscore', 'average_forever', 'median_forever', 'release_date', 'english', 'platforms',
       'required_age', 'categories', 'genres', 'steamspy_tags', 'achievements', 'positive_ratings',
       'negative_ratings', 'average_playtime', 'median_playtime', 'owners', 'price'],
      dtype='object')

Adding to this newly created data frame i think that it would be useful to include the game descriptions of each of these video games.

In [226]:
# checking the data frame containing all game descriptions
game_desc

,steam_appid,detailed_description,about_the_game,short_description
0,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...
1,20,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...
2,30,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...
3,40,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...
4,50,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...
...,...,...,...,...
27329,1065230,"<img src=""https://steamcdn-a.akamaihd.net/stea...","<img src=""https://steamcdn-a.akamaihd.net/stea...",The Room of Pandora is a third-person interact...
27330,1065570,Have you ever been so lonely that no one but y...,Have you ever been so lonely that no one but y...,Cyber Gun is a hardcore first-person shooter w...
27331,1065650,<strong>Super Star Blast </strong>is a space b...,<strong>Super Star Blast </strong>is a space b...,Super Star Blast is a space based game with ch...
27332,1066700,Pursue a snow-white deer through an enchanted ...,Pursue a snow-white deer through an enchanted ...,Pursue a snow-white deer through an enchanted ...


In [227]:
# renaming steam_appid to appid in order to merge the data frames together
game_desc.rename(columns={'steam_appid': 'appid'}, inplace=True)

In [228]:
# merging the game_info data frame to game_desc data frame
game_info = pd.DataFrame.merge(game_info, game_desc,on='appid')

In [229]:
# displaying the updated data frame
game_info.head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,average_forever,median_forever,release_date,english,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,detailed_description,about_the_game,short_description
0,570,Dota 2,Valve,Valve,,1096833,194259,0,31068,1411,2013-07-09,1,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,863507,142079,23944,801,100000000-200000000,0.00,<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter..."
1,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,,3834378,533122,0,24958,7370,2012-08-21,1,windows;mac;linux,0,Multi-player;Steam Achievements;Full controlle...,Action;Free to Play,FPS;Multiplayer;Shooter,167,2644404,402313,22494,6502,50000000-100000000,0.00,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...
2,578080,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,PUBG Corporation,,781419,664521,0,23310,10668,2017-12-21,1,windows,0,Multi-player;Online Multi-Player;Stats,Action;Adventure;Massively Multiplayer,Survival;Shooter;Multiplayer,37,496184,487076,22938,12434,50000000-100000000,26.99,<strong>PLAYERUNKNOWN'S BATTLEGROUNDS</strong>...,<strong>PLAYERUNKNOWN'S BATTLEGROUNDS</strong>...,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...
3,440,Team Fortress 2,Valve,Valve,,627500,39186,0,9705,409,2007-10-10,1,windows;mac;linux,0,Multi-player;Cross-Platform Multiplayer;Steam ...,Action;Free to Play,Free to Play;Multiplayer;FPS,520,515879,34036,8495,623,20000000-50000000,0.00,"<h1>The Jungle Inferno Update</h1><p><a href=""...","<p><strong>""The most fun you can have online""<...",Nine distinct classes provide a broad range of...
4,304930,Unturned,Smartly Dressed Games,Smartly Dressed Games,,349238,34817,0,2514,369,2017-07-07,1,windows;mac;linux,0,Single-player;Online Multi-Player;Online Co-op...,Action;Adventure;Casual;Free to Play;Indie,Free to Play;Survival;Zombies,46,292574,31482,3248,413,20000000-50000000,0.00,"<img src=""https://steamcdn-a.akamaihd.net/stea...","<img src=""https://steamcdn-a.akamaihd.net/stea...",You're a survivor in the zombie infested ruins...


Using the updated game_info data frame i will be adding it to the player information data frame so that 

In [230]:
# checking the player information data frame
purchase_play_info

,0,1,2,3,4
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0
2,151603712,Fallout 4,purchase,1.0,0
3,151603712,Fallout 4,play,87.0,0
4,151603712,Spore,purchase,1.0,0
...,...,...,...,...,...
199995,128470551,Titan Souls,play,1.5,0
199996,128470551,Grand Theft Auto Vice City,purchase,1.0,0
199997,128470551,Grand Theft Auto Vice City,play,1.5,0
199998,128470551,RUSH,purchase,1.0,0


In [231]:
# renaming columns
purchase_play_info.rename(columns={0: 'id', 1: 'name', 2: 'purchase',
                     3: 'hours_of_play'}, inplace=True)

In [232]:
# displaying changes
purchase_play_info.head()

,id,name,purchase,hours_of_play,4
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0
2,151603712,Fallout 4,purchase,1.0,0
3,151603712,Fallout 4,play,87.0,0
4,151603712,Spore,purchase,1.0,0


In [233]:
# checking column names
purchase_play_info.columns

Index(['id', 'name', 'purchase', 'hours_of_play', 4], dtype='object')

In [234]:
# checking what information column 4 contains within it
purchase_play_info[4].unique()

array([0], dtype=int64)

In [235]:
# as column 4 conains soley zeros, this column will be removed
del purchase_play_info[4]

In [236]:
# displaying the changed data frame
purchase_play_info.head()

,id,name,purchase,hours_of_play
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0
1,151603712,The Elder Scrolls V Skyrim,play,273.0
2,151603712,Fallout 4,purchase,1.0
3,151603712,Fallout 4,play,87.0
4,151603712,Spore,purchase,1.0


In [237]:
# checking player information based on a specific game
purchase_play_info[purchase_play_info.name == 'Dota 2']

,id,name,purchase,hours_of_play
42,151603712,Dota 2,purchase,1.0
43,151603712,Dota 2,play,0.5
66,187131847,Dota 2,purchase,1.0
67,187131847,Dota 2,play,2.3
855,176410694,Dota 2,purchase,1.0
...,...,...,...,...
199897,99096740,Dota 2,play,1704.0
199948,176449171,Dota 2,purchase,1.0
199949,176449171,Dota 2,play,1310.0
199960,221315846,Dota 2,purchase,1.0


In [238]:
# merging the two data frames into one based on the name of the game column
df_final = pd.DataFrame.merge(game_info, purchase_play_info, on='name')

In [239]:
# previewing the final data frame
df_final.head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,average_forever,median_forever,release_date,english,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,detailed_description,about_the_game,short_description,id,purchase,hours_of_play
0,570,Dota 2,Valve,Valve,,1096833,194259,0,31068,1411,2013-07-09,1,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,863507,142079,23944,801,100000000-200000000,0.0,<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...",151603712,purchase,1.0
1,570,Dota 2,Valve,Valve,,1096833,194259,0,31068,1411,2013-07-09,1,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,863507,142079,23944,801,100000000-200000000,0.0,<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...",151603712,play,0.5
2,570,Dota 2,Valve,Valve,,1096833,194259,0,31068,1411,2013-07-09,1,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,863507,142079,23944,801,100000000-200000000,0.0,<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...",187131847,purchase,1.0
3,570,Dota 2,Valve,Valve,,1096833,194259,0,31068,1411,2013-07-09,1,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,863507,142079,23944,801,100000000-200000000,0.0,<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...",187131847,play,2.3
4,570,Dota 2,Valve,Valve,,1096833,194259,0,31068,1411,2013-07-09,1,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,863507,142079,23944,801,100000000-200000000,0.0,<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...",176410694,purchase,1.0


In [240]:
# checking its shape
df_final.shape

(99632, 30)

# DATA CLEANING

In [241]:
# renaming the final data frame
steam = df_final

### Checking for null values

In [242]:
# detecting nan values in dataframe
steam.isnull().values.any()

False

In [243]:
# locating where the nan values exist and the amount of nan values for each columns
steam.isnull().sum()

appid                   0
name                    0
developer               0
publisher               0
score_rank              0
positive                0
negative                0
userscore               0
average_forever         0
median_forever          0
release_date            0
english                 0
platforms               0
required_age            0
categories              0
genres                  0
steamspy_tags           0
achievements            0
positive_ratings        0
negative_ratings        0
average_playtime        0
median_playtime         0
owners                  0
price                   0
detailed_description    0
about_the_game          0
short_description       0
id                      0
purchase                0
hours_of_play           0
dtype: int64

From these results there have been no null values found.

In [244]:
# checking the shape of the data frame
steam.shape

(99632, 30)

In [245]:
# checking the data types
steam.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99632 entries, 0 to 99631
Data columns (total 30 columns):
appid                   99632 non-null object
name                    99632 non-null object
developer               99632 non-null object
publisher               99632 non-null object
score_rank              99632 non-null object
positive                99632 non-null object
negative                99632 non-null object
userscore               99632 non-null object
average_forever         99632 non-null object
median_forever          99632 non-null object
release_date            99632 non-null object
english                 99632 non-null int64
platforms               99632 non-null object
required_age            99632 non-null int64
categories              99632 non-null object
genres                  99632 non-null object
steamspy_tags           99632 non-null object
achievements            99632 non-null int64
positive_ratings        99632 non-null int64
negative_ratings     

In [246]:
# checking min, max and other relevant information
steam.describe()

,english,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,price,id,hours_of_play
count,99632.000000,99632.000000,99632.000000,99632.000000,99632.000000,99632.000000,99632.000000,99632.000000,9.963200e+04,99632.000000
mean,0.999689,2.189136,73.792958,158305.744098,21695.036173,4445.184629,589.851333,6.667732,1.135632e+08,21.761117
std,0.017637,5.763453,155.461837,264246.799053,42467.627565,7271.300668,1439.313553,7.554852,7.539969e+07,172.208015
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.250000e+03,0.100000
25%,1.000000,0.000000,0.000000,3700.000000,657.500000,258.000000,150.000000,0.000000,5.108578e+07,1.000000
50%,1.000000,0.000000,29.000000,27755.000000,2609.000000,949.000000,362.000000,6.990000,1.012121e+08,1.000000
75%,1.000000,0.000000,67.000000,144595.000000,16433.000000,4760.000000,801.000000,9.990000,1.668631e+08,2.000000
max,1.000000,18.000000,1746.000000,863507.000000,142079.000000,95242.000000,190445.000000,69.990000,3.099031e+08,11754.000000


In [247]:
# converting release_date to date time format
steam['release_date'] = pd.to_datetime(steam['release_date'])

In [248]:
# displaying data frame
steam.head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,average_forever,median_forever,release_date,english,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,detailed_description,about_the_game,short_description,id,purchase,hours_of_play
0,570,Dota 2,Valve,Valve,,1096833,194259,0,31068,1411,2013-07-09,1,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,863507,142079,23944,801,100000000-200000000,0.0,<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...",151603712,purchase,1.0
1,570,Dota 2,Valve,Valve,,1096833,194259,0,31068,1411,2013-07-09,1,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,863507,142079,23944,801,100000000-200000000,0.0,<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...",151603712,play,0.5
2,570,Dota 2,Valve,Valve,,1096833,194259,0,31068,1411,2013-07-09,1,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,863507,142079,23944,801,100000000-200000000,0.0,<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...",187131847,purchase,1.0
3,570,Dota 2,Valve,Valve,,1096833,194259,0,31068,1411,2013-07-09,1,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,863507,142079,23944,801,100000000-200000000,0.0,<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...",187131847,play,2.3
4,570,Dota 2,Valve,Valve,,1096833,194259,0,31068,1411,2013-07-09,1,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,863507,142079,23944,801,100000000-200000000,0.0,<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...",176410694,purchase,1.0


In [249]:
# checking the current arrangement of the columns
steam.columns

Index(['appid', 'name', 'developer', 'publisher', 'score_rank', 'positive', 'negative',
       'userscore', 'average_forever', 'median_forever', 'release_date', 'english', 'platforms',
       'required_age', 'categories', 'genres', 'steamspy_tags', 'achievements', 'positive_ratings',
       'negative_ratings', 'average_playtime', 'median_playtime', 'owners', 'price',
       'detailed_description', 'about_the_game', 'short_description', 'id', 'purchase',
       'hours_of_play'],
      dtype='object')

In [250]:
# changing the arrangment of the columns
steam = steam[['id', 'appid', 'name', 'purchase', 'hours_of_play', 'developer',
       'publisher', 'score_rank', 'positive', 'negative', 'userscore', 'release_date', 'english',
       'platforms', 'required_age', 'categories', 'genres', 'steamspy_tags', 'achievements',
       'positive_ratings', 'negative_ratings', 'average_playtime', 'average_forever',
       'median_playtime', 'median_forever', 'owners', 'detailed_description', 'about_the_game',
       'short_description', 'price']]

In [251]:
# checking the changes made
steam.columns

Index(['id', 'appid', 'name', 'purchase', 'hours_of_play', 'developer', 'publisher', 'score_rank',
       'positive', 'negative', 'userscore', 'release_date', 'english', 'platforms', 'required_age',
       'categories', 'genres', 'steamspy_tags', 'achievements', 'positive_ratings',
       'negative_ratings', 'average_playtime', 'average_forever', 'median_playtime',
       'median_forever', 'owners', 'detailed_description', 'about_the_game', 'short_description',
       'price'],
      dtype='object')

In [252]:
# displaying the data frame to check the changes made
steam.head()

,id,appid,name,purchase,hours_of_play,developer,publisher,score_rank,positive,negative,userscore,release_date,english,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,average_forever,median_playtime,median_forever,owners,detailed_description,about_the_game,short_description,price
0,151603712,570,Dota 2,purchase,1.0,Valve,Valve,,1096833,194259,0,2013-07-09,1,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,863507,142079,23944,31068,801,1411,100000000-200000000,<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...",0.0
1,151603712,570,Dota 2,play,0.5,Valve,Valve,,1096833,194259,0,2013-07-09,1,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,863507,142079,23944,31068,801,1411,100000000-200000000,<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...",0.0
2,187131847,570,Dota 2,purchase,1.0,Valve,Valve,,1096833,194259,0,2013-07-09,1,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,863507,142079,23944,31068,801,1411,100000000-200000000,<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...",0.0
3,187131847,570,Dota 2,play,2.3,Valve,Valve,,1096833,194259,0,2013-07-09,1,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,863507,142079,23944,31068,801,1411,100000000-200000000,<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...",0.0
4,176410694,570,Dota 2,purchase,1.0,Valve,Valve,,1096833,194259,0,2013-07-09,1,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,863507,142079,23944,31068,801,1411,100000000-200000000,<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...",0.0


In [253]:
# checking the data types
steam.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99632 entries, 0 to 99631
Data columns (total 30 columns):
id                      99632 non-null int64
appid                   99632 non-null object
name                    99632 non-null object
purchase                99632 non-null object
hours_of_play           99632 non-null float64
developer               99632 non-null object
publisher               99632 non-null object
score_rank              99632 non-null object
positive                99632 non-null object
negative                99632 non-null object
userscore               99632 non-null object
release_date            99632 non-null datetime64[ns]
english                 99632 non-null int64
platforms               99632 non-null object
required_age            99632 non-null int64
categories              99632 non-null object
genres                  99632 non-null object
steamspy_tags           99632 non-null object
achievements            99632 non-null int64
positive_rat

In [254]:
# converting id to an interger
steam['id'].astype(str).astype(int)

0        151603712
1        151603712
2        187131847
3        187131847
4        176410694
           ...    
99627     11373749
99628     55906572
99629     17530772
99630    192170147
99631    154230723
Name: id, Length: 99632, dtype: int32

In [255]:
# converting appid to an integer
steam['appid'] = steam['appid'].astype(str).astype('int64')

In [256]:
# checking if the changes were made
steam.dtypes

id                               int64
appid                            int64
name                            object
purchase                        object
hours_of_play                  float64
developer                       object
publisher                       object
score_rank                      object
positive                        object
negative                        object
userscore                       object
release_date            datetime64[ns]
english                          int64
platforms                       object
required_age                     int64
categories                      object
genres                          object
steamspy_tags                   object
achievements                     int64
positive_ratings                 int64
negative_ratings                 int64
average_playtime                 int64
average_forever                 object
median_playtime                  int64
median_forever                  object
owners                   

In [257]:
# checking the unique game names in the column 'name'
steam.name.unique()

array(['Dota 2', 'Team Fortress 2', 'Unturned', ..., 'Project Aftermath',
       "Putt-Putt and Pep's Dog on a Stick", "YOU DON'T KNOW JACK MOVIES"],
      dtype=object)

In [258]:
# checking the unique game names in the column 'score_rank'
steam.score_rank.unique()

array([''], dtype=object)

In [259]:
# checking the unique game names in the column 'userscore'
steam.userscore.unique()

array([0], dtype=object)

As both 'score_rank' and 'userscore' contain variables that are empty or contain zeros, these columns will be deleted as they are not usefull. In addition to that 'average_forever', 'median_forever', 'postivie_ratings' and 'negative_ratings' will also be removed from the data frame.

In [260]:
# deleting the unessesary columns
del steam['userscore']
del steam['score_rank']
del steam['average_forever']
del steam['median_forever']
del steam['positive_ratings']
del steam['negative_ratings']

In [261]:
# creating a new column to show the percentage of positive ratings of games
# adding the positive and negative columns
steam['rank'] = steam['positive'] + steam['negative']

In [262]:
# dividing positive column by the rank column
steam['rank'] = steam['positive']/steam['rank']

In [263]:
# multiplying rank by 100 to get percentage
steam['rank'] = steam['rank'] * 100

# Final data frame

<b>Variables:</b>
<li><b>'id':</b> unique ids of users
<li><b>'appid':</b> unique ids of each game
<li><b>'name':</b> name of the game
<li><b>'purchase':</b> purchased or played
<li><b>'hours_of_play':</b> hours of play, value is 1.0 if purchased   
<li><b>'developer':</b> developer of the game
<li><b>'publisher':</b> publisher of the game
<li><b>'positive':</b> postive ratings of the game
<li><b>'negative':</b> negative ratings of the game
<li><b>'release_date':</b> game release date
<li><b>'english':</b> returns if the game is english or not
<li><b>'platforms':</b> compatible platforms
<li><b>'required_age':</b> minimum age requirements of game
<li><b>'categories':</b> game categories
<li><b>'genres':</b> game genres
<li><b>'steamspy_tags':</b> tags of game
<li><b>'achievements':</b> game achievements completed
<li><b>'average_playtime':</b> average playtime
<li><b>'median_playtime':</b> median playtime
<li><b>'owners':</b> amount of people who purpchased the game
<li><b>'detailed_description':</b> detailed description of the game
<li><b>'about_the_game':</b> information about the game
<li><b>'short_description':</b> short description of the game
<li><b>'price':</b> price
<li><b>'rank':</b> percentage of positive ratings

In [264]:
# checking the final data frame
steam.head()

,id,appid,name,purchase,hours_of_play,developer,publisher,positive,negative,release_date,english,platforms,required_age,categories,genres,steamspy_tags,achievements,average_playtime,median_playtime,owners,detailed_description,about_the_game,short_description,price,rank
0,151603712,570,Dota 2,purchase,1.0,Valve,Valve,1096833,194259,2013-07-09,1,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,23944,801,100000000-200000000,<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...",0.0,84.9539
1,151603712,570,Dota 2,play,0.5,Valve,Valve,1096833,194259,2013-07-09,1,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,23944,801,100000000-200000000,<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...",0.0,84.9539
2,187131847,570,Dota 2,purchase,1.0,Valve,Valve,1096833,194259,2013-07-09,1,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,23944,801,100000000-200000000,<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...",0.0,84.9539
3,187131847,570,Dota 2,play,2.3,Valve,Valve,1096833,194259,2013-07-09,1,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,23944,801,100000000-200000000,<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...",0.0,84.9539
4,176410694,570,Dota 2,purchase,1.0,Valve,Valve,1096833,194259,2013-07-09,1,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Strategy,0,23944,801,100000000-200000000,<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...",0.0,84.9539


# Saving the final cleaned data frame to a csv file

In [265]:
steam.to_csv('steam_rs.csv', index=False)